# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [40]:
%%writefile ../flows/cloud/taxi_data_champion.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger
from metaflow import project, S3
from metaflow.cards import Markdown, Table, Image, Artifact

# URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
URL = 's3://outerbounds-datasets/taxi/latest.parquet'

DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2'})
@project(name='taxifare_prediction')
class TaxiFarePrediction(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):
       

        obviously_bad_data_filters = [
            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.passenger_count <= 5 #most taxis can only sit 4 passengers
        ]
        for f in obviously_bad_data_filters:
            df = df[f]
            
        return df
    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        with S3() as s3:
            obj = s3.get(URL)
            df = pd.read_parquet(obj.path)

        self.df = self.transform_features(df)
        # self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
        
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)
    
    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import LinearRegression

        # TODO: Play around with the model if you are feeling it.
        self.model = LinearRegression()
        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        self.model_name = 'champion'
        self.model_type = "regression"
        print(f'Score: {self.scores.mean():.2f}')
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ../flows/cloud/taxi_data_champion.py


In [4]:
! python ../flows/cloud/taxi_data_champion.py --environment=conda run

Metaflow 2.8.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxifare_prediction, Branch: user.sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Bootstrapping conda environment...(this could take a few minutes)
2023-05-24 05:14:31.226 Workflow starting (run-id 386), see it in the UI at https://ui-pw-2141923211.outerbounds.dev/TaxiFarePrediction/386
2023-05-24 05:14:32.200 [386/start/961 (pid 18590)] Task is starting.
2023-05-24 05:14:43.526 [386/start/961 (pid 18590)] Task finished successfully.
2023-05-24 05:14:44.477 [386/linear_model/962 (pid 18701)] Task is starting.
2023-05-24 05:14:46.934 [386/linear_model/962 (pid 18701)] Task finished successfully.
2023-05-24 05:14:47.978 [386/validate/963 (pid 18745)] Task is starting.
2023-05-24 05:14:54.689 [386/validate/963 (pid 18745)] Task finished successfully.
2023-05-24 05:14:55.642 [386/end/964 (pid 18809)] Task is starting.
2023-05-24 05:14:56.835 [386/end/964 (pid 18809)

In [11]:
#create workflow
! python ../flows/cloud/taxi_data_champion.py --environment=conda --production --branch champion --production argo-workflows create

Metaflow 2.8.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxifare_prediction, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction.prod.champion.taxifareprediction to Argo Workflows...

The namespace of this production flow is
    production:mfprj-bl5jeer7xehnsiac-0-hqum
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-bl5jeer7xehnsiac-0-hqum")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-bl5jeer7xehnsiac-0-hqum
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping conda environment...(this could take a few minutes)
Workflow taxifareprediction.prod.champion.taxifareprediction for flow TaxiFar

In [12]:
#trigger workflow
! python ../flows/cloud/taxi_data_champion.py --environment=conda --production --branch champion --production argo-workflows trigger

Metaflow 2.8.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxifare_prediction, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taxifareprediction.prod.champion.taxifareprediction triggered on Argo Workflows (run-id argo-taxifareprediction.prod.champion.taxifareprediction-nnzbz).
See the run in the UI at https://ui-pw-2141923211.outerbounds.dev/TaxiFarePrediction/argo-taxifareprediction.prod.champion.taxifareprediction-nnzbz


## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

In [38]:
%%writefile ../flows/cloud/taxi_data_challenger.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, Flow, trigger
from metaflow import project, S3
from metaflow.cards import Markdown, Table, Image, Artifact

# URL = "https://outerbounds-datasets.s3.us-west-2.amazonaws.com/taxi/latest.parquet"
URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2', 'xgboost' : '1.7.4'})
@project(name='taxifare_prediction')
class TaxiFarePrediction(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):
        # TODO: 
        # Try to complete tasks 2 and 3 with this function doing nothing like it currently is.
        # Understand what is happening.
        # Revisit task 1 and think about what might go in this function.

        obviously_bad_data_filters = [
            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.passenger_count <= 5 #most taxis can only sit 4 passengers
        ]
        for f in obviously_bad_data_filters:
            df = df[f]

        return df

    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        with S3() as s3:
            obj = s3.get(URL)
            df = pd.read_parquet(obj.path)

        self.df = self.transform_features(df)
        # self.df = self.transform_features(pd.read_parquet(self.data_url))

        # NOTE: we are split into training and validation set in the validation step which uses cross_val_score.
      
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.xgboost_model)
    
    @step
    def xgboost_model(self):
        "Fit an XGBoost regressor"
        from xgboost import XGBRegressor
        self.model = XGBRegressor()
        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows

    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score

        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        self.model_name = 'challenger'
        self.model_type = "xgboost"
        print(f'{self.model_name} score: {self.scores.mean():.2f}')
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ../flows/cloud/taxi_data_challenger.py


In [9]:
! python ../flows/cloud/taxi_data_challenger.py --environment=conda run

Metaflow 2.8.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxifare_prediction, Branch: user.sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Bootstrapping conda environment...(this could take a few minutes)
2023-05-24 05:25:32.319 Workflow starting (run-id 388), see it in the UI at https://ui-pw-2141923211.outerbounds.dev/TaxiFarePrediction/388
2023-05-24 05:25:33.275 [388/start/974 (pid 19248)] Task is starting.
2023-05-24 05:25:40.012 [388/start/974 (pid 19248)] Task finished successfully.
2023-05-24 05:25:40.946 [388/xgboost_model/975 (pid 19344)] Task is starting.
2023-05-24 05:25:43.739 [388/xgboost_model/975 (pid 19344)] Task finished successfully.
2023-05-24 05:25:44.614 [388/validate/976 (pid 19396)] Task is starting.
2023-05-24 05:27:11.461 [388/validate/976 (pid 19396)] Task finished successfully.
2023-05-24 05:27:12.395 [388/end/977 (pid 19459)] Task is starting.
2023-05-24 05:27:13.928 [388/end/977 (pid 1945

In [13]:
! python ../flows/cloud/taxi_data_challenger.py --environment=conda --production --branch challenger --production argo-workflows create

Metaflow 2.8.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxifare_prediction, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying taxifareprediction.prod.challenger.taxifareprediction to Argo Workflows...
It seems this is the first time you are deploying taxifareprediction.prod.challenger.taxifareprediction to Argo Workflows.

A new production token generated.

The namespace of this production flow is
    production:mfprj-a2atl4ukaehxrfie-0-uate
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-a2atl4ukaehxrfie-0-uate")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-a2atl4ukaehxrfie-0-uate
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about p

In [14]:
#trigger workflow
! python ../flows/cloud/taxi_data_challenger.py --environment=conda --production --branch challenger --production argo-workflows trigger

Metaflow 2.8.6+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: taxifare_prediction, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow taxifareprediction.prod.challenger.taxifareprediction triggered on Argo Workflows (run-id argo-taxifareprediction.prod.challenger.taxifareprediction-gn9wk).
See the run in the UI at https://ui-pw-2141923211.outerbounds.dev/TaxiFarePrediction/argo-taxifareprediction.prod.challenger.taxifareprediction-gn9wk


## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  

I would need to monitor data quality issues to get buy-in for the challenger model. Currently, the data is hardcoded to exclude values using the obviously_bad_filters function. With domain expertise, this could be customized even further, with a cascading sequence for handling missing completely at random vs missing not at random values. 

In [31]:
from metaflow import namespace, Metaflow

namespace('user:sandbox')
Metaflow().flows

[Flow('Template_Flow'),
 Flow('TFlow1'),
 Flow('RF_Flow'),
 Flow('BSTFlow'),
 Flow('NNFlow'),
 Flow('BaselineNLPFlow'),
 Flow('Branch_Flow'),
 Flow('TFlow2'),
 Flow('TFlow4'),
 Flow('TFlow5'),
 Flow('BaselineChallenge'),
 Flow('BaselineChallenge_1'),
 Flow('RF_Flow_cloud'),
 Flow('Branch_Flow_Cloud'),
 Flow('Branch_Cloud_Step'),
 Flow('Branch_Cloud_Flow'),
 Flow('DivideByZeroFlow'),
 Flow('RetryFlow'),
 Flow('TimeoutFlow'),
 Flow('CatchDivideByZeroFlow'),
 Flow('TaxiFarePrediction'),
 Flow('TitanicSurvivalPredictor')]

In [44]:
from metaflow import Flow, namespace

# these values are unique to your deployment!
CHAMPION_MODEL_NAMESPACE = 'production:mfprj-bl5jeer7xehnsiac-0-hqum' # "production:mfprj-xsfdb3gtsiboqyrd-0-vqsy"
CHALLENGER_MODEL_NAMESPACE = 'production:mfprj-a2atl4ukaehxrfie-0-uate' # "production:mfprj-cfyzlfzievjlmsf4-0-tbgz"

best_score = -1; winner = None; winner_namespace = None
for n in [CHAMPION_MODEL_NAMESPACE, CHALLENGER_MODEL_NAMESPACE]:
    namespace(n)
    run = Flow('TaxiFarePrediction').latest_successful_run
    acc_score = max(run.data.scores)
    if acc_score > best_score:
        best_score = acc_score
        winner = run.data.model_name
        winner_namespace = n

In [45]:
print("The winner is the {} model, with accuracy of {}%. You can find the model in the flow deployed to the {} namespace.".format(winner, round(100*best_score, 2), winner_namespace))

The winner is the challenger model, with accuracy of 90.78%. You can find the model in the flow deployed to the production:mfprj-a2atl4ukaehxrfie-0-uate namespace.


## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!